In [1]:
import numpy as np
import random

In [2]:
# binary spliting
def binary_splitting_round(s):
    # s: np.array the infectious status & test status
    num = 0
    flag = sum(s[:,0])>0
    assert flag
    stages = 0
    if len(s[:,0])==1:
        s[0,1] = s[0,0]
        return num,s,stages
    
    B1, B2 = np.array_split(s.copy(), 2,axis=0)
    flag = sum(B1[:,0])>0
    num+=1
    stages += 1
    
    if flag:
        n,stmp,stage = binary_splitting_round(B1)
        s[:len(B1),1] = stmp[:,1]
    else:
        s[:len(B1),1] = 0
        n,stmp,stage = binary_splitting_round(B2)
        s[len(B1):,1] = stmp[:,1]
    num += n
    stages += stage
    return num,s,stages 

def binary_splitting(s):
    # modified bs
    # s: 1-d array the infectious status
    st = np.zeros((len(s),2))
    st[:,0] = s
    st[:,1] = np.nan
    nums = 0
    count = sum(np.isnan(st[:,1]))
    stages = 0
    # the undetermined people
    while count!=0:
        mask = np.isnan(st[:,1])
        flag = sum(st[mask,0]>0)>0
        nums += 1
        stages+=1
        if not flag:
            st[mask,1] = 0
        else:
            n,stmp,stage = binary_splitting_round(st[mask,:])
            st[mask,1] = stmp[:,1]
            nums += n
            stages += stage
        count = sum(np.isnan(st[:,1]))
        
    assert sum(st[:,0]!=st[:,1])==0
    return nums,stages, st[:,1]

# diag
def diagalg_iter(s):
    # s(np.array): binary string of infection status
    k = int(np.log2(len(s)))
    l = int(2**(k-1))
    lp = 0
    p = np.zeros(k+1)
    group = dict()
    num = np.ones(k+1,dtype=np.int32)
    for i in range(k):
        p[i] = sum(s[lp:lp+l])>0
        group[i] = s[lp:lp+l]
        num[i] = l
        lp+=l
        l = l//2

    p[-1] = s[-1]
    group[k] = np.array([s[-1]])
    # p(array): pattern
    # group(dict): indicate the group information
    # num(array): the group size
    return p.astype(np.int32), group,num


def diag_splitting(s):
    # s(np.array): binary string of infection status
    num_tests = 0
    stages = 0
    pattern, group, nums = diagalg_iter(s)
    stages +=1
    num_tests += len(pattern)
    indices = np.where(pattern == 1)[0]
    flag = 0
    for i in indices:
        if nums[i]>1:
            num_test,stage = diag_splitting(group[i])
            num_tests += num_test
            if not flag:
                stages+=stage
                flag = 1
    return num_tests,stages

In [ ]:
m_arr = [0, 1, 0, 1, 0, 0, 1, 0, 1, 0]
# should be: 7 tests, 3 stages w/o binary splitting
t, s, arr =  binary_splitting(m_arr)
print("Tests: ", t, "\nStages: ", s, "\nReturn array: ", arr)

In [3]:
import numpy as np

def SBM(N,M,q0,q1):
    '''This function is designed to generate the Stochastic Block Model.
    input params (consistent with the project description):
    N (int): the number of individuals
    M (int): the number of subgroups
    q0 (float): probability of inter-subgroup connection
    q1 (float): probability of intra-subgroup connection

    output:
    G (N*N): adjacency matrix of the generated graph
    '''
    #################################################
    ''' your code here'''
    G = np.zeros((N,N), dtype=int)

    '''
    i % size = index into subgroup
    '''

    size = round(N/M)
    
    #Testing
    #print("Group size: ", size)

   #  '''
    for i in range(N):
        i_subgroup = np.floor((i)/size)       # which subgroup this index is in
        for j in range(N):
            if i == j:
                continue
            
            j_subgroup = np.floor((j)/size)
            
            if (j_subgroup == i_subgroup):
                connected = np.random.choice([0,1], p=[1-q0, q0])   # for inter-group
            else:
                connected = np.random.choice([0,1], p=[1-q1, q1])   # for intra-group

            G[i][j] = connected
            G[j][i] = connected
    # '''
    #################################################

    return G

In [4]:
def infect_step(G,p1,individuals,N):
    '''The function serves as the infection model for each day.
    input params (consistent with the project description):

    
    G (ndarray N*N): the adjacency matrix.
    p1: the probability each individual infects neighbours.
    individuals = who's already infected
    N = # of people in 'individuals'
    '''

    ###################################################
    '''your code here'''
    individuals_updated = np.copy(individuals)
    for i in range(N):
        if individuals[i] == 1:     # this individual is infected
            ''' Need to find all of their neighbors and infect them w/
                probability p1 '''
            for neighbor_indx in range(N):
                if G[i][neighbor_indx] == 1:    # if i is neighbors w/ neighbor_indx

                    if individuals_updated[neighbor_indx] == 0: # if this person is NOT already infected
                        individuals_updated[neighbor_indx] = np.random.choice([0,1], p=[1-p1, p1])

                    # for debugging purposes:
                    #if individuals_updated[neighbor_indx] == 1:
                     #   print("\tInfecting ", neighbor_indx, ", who is a neighbor of ", i)
        
    ###################################################
    return individuals_updated


In [5]:
def infect(G,p0,p1,time_steps):
    '''The function serves as the infection model for each day.
    input params (consistent with the project description):
    G (ndarray N*N): the adjacency matrix.
    p0: the infection probability for initial status.
    p1: the probability each individual infects neighbours.
    time_steps: log N
    '''
    N = G.shape[0]
    individuals = np.zeros(N)
    ###################################################
    '''your code here'''
    for i in range(N):
        # infect everyone w/ initial probability p0
        individuals[i] = np.random.choice([0,1], p=[1-p0, p0])
    print("Original infected individuals: ", individuals)
    
    for _ in range(time_steps):
        individuals = infect_step(G, p1, individuals, N)
        print("Infected individuals after step ", _, " - ", individuals)
        
    ###################################################

    return individuals

In [6]:
def num_infected(s):
    inf_people = 0
    for i in range(len(s)):
        inf_people += s[i]
    return inf_people

In [9]:
m_arr = [0, 1, 0, 1, 0, 0, 1, 0, 1, 0]
# should be: 7 tests, 3 stages w/o binary splitting
lisa(m_arr)

RUNNING TEST ON original:  [0, 1, 0, 1, 0, 0, 1, 0, 1, 0] 


TESTING group:  [0, 1, 0, 1, 0, 0, 1, 0, 1, 0]

TESTING group:  [0, 1]
--Num binary tests:  2
--Num binary stages:  2
Setting stages to  4

TESTING group:  [0, 1]
--Num binary tests:  2
--Num binary stages:  2
Setting stages to  4

TESTING group:  [0, 0]
0 infected --> ran only 1 test
Setting stages to  4

TESTING group:  [1, 0, 1, 0]

TESTING group:  [1, 0]
--Num binary tests:  3
--Num binary stages:  3
Setting stages to  5

TESTING group:  [1, 0]
--Num binary tests:  3
--Num binary stages:  3
Setting stages to  5
Setting stages to  6


(17, 6)

In [125]:
m_arr2 = [0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0]
lisa(m_arr2)

RUNNING TEST ON original:  [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0] 


	Test no:  0
Number of infected people:  4
s subgroup:  [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]

	Test no:  0
Number of infected people:  0
s subgroup:  [0, 0, 0, 0]
INCREASING stages by  0  -- current subgroup:  [0, 0, 0, 0]

	Test no:  1
Number of infected people:  1
s subgroup:  [1, 0, 0, 0]
#### Switching to binary splitting b/c only one infected person
--Binary splitting tests:  4
--Binary splitting stages:  4
INCREASING stages by  4  -- current subgroup:  [1, 0, 0, 0]

	Test no:  5
Number of infected people:  2
s subgroup:  [0, 1, 0, 1]

	Test no:  5
Number of infected people:  1
s subgroup:  [0, 1]
#### Switching to binary splitting b/c only one infected person
--Binary splitting tests:  2
--Binary splitting stages:  2
INCREASING stages by  2  -- current subgroup:  [0, 1]

	Test no:  7
Number of infected people:  1
s subgroup:  [0, 1]
#### Switching to binary splitting b/c o

(19, 8)

In [7]:
def mona(s, max_stages):
    inf_people = num_infected(s)
    print("\nTESTING group: ", s)
  

    if (inf_people  == 1):
        binary_tests, binary_stages, _ = binary_splitting(s)
        print("--Num binary tests: ", binary_tests)
        print("--Num binary stages: ", binary_stages)
        if len(s) == 1:
            return 1,1
        else:
            return 1 + binary_tests, 1 + binary_stages
    elif(inf_people < 1):
        print("0 infected --> ran only 1 test")
        return 1, 1
    else:
        num_per_group = round(len(s)/inf_people)
        
        # for each subgroup, call function again
        tests = 1 # = this testing group
        for i in range(inf_people):
            stages = 1
            if(i == inf_people-1):
                subGroup = s[i*num_per_group:]
            else:
                subGroup = s[i*num_per_group:(i+1)*(num_per_group)]
            
            t, st = mona(subGroup, max_stages)
            stages += st
            tests += t   # number of tests in subtree

            print("Setting stages to ", max(max_stages, stages))
            max_stages = max(max_stages, stages)
        return tests, max_stages
 
def lisa(s):
    '''
    s(np.array): binary string of infection status
    '''
    num_tests = 0
    stages = 0
    ###################################################
    '''your code here'''
    print("RUNNING TEST ON original: ", s, "\n")
    num_tests, stages = mona(s, stages)

    ###################################################

    return num_tests, stages

m_arr4 = [0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
lisa(m_arr4)


RUNNING TEST ON original:  [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 


TESTING group:  [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

TESTING group:  [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]

TESTING group:  [0, 0, 0, 0, 1, 0, 0]
--Num binary tests:  5
--Num binary stages:  5
Setting stages to  7

TESTING group:  [0, 0, 1, 0, 0, 0, 0]
--Num binary tests:  5
--Num binary stages:  5
Setting stages to  7
Setting stages to  8

TESTING group:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0 infected --> ran only 1 test
Setting stages to  8


(15, 8)

In [8]:
m_arr3 = [0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0]
lisa(m_arr3)

RUNNING TEST ON original:  [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0] 


TESTING group:  [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]

TESTING group:  [0, 0, 0, 0, 1]
--Num binary tests:  3
--Num binary stages:  3
Setting stages to  5

TESTING group:  [0, 0, 0, 0, 1]
--Num binary tests:  3
--Num binary stages:  3
Setting stages to  5

TESTING group:  [0, 1, 0, 1, 0]

TESTING group:  [0, 1]
--Num binary tests:  2
--Num binary stages:  2
Setting stages to  5

TESTING group:  [0, 1, 0]
--Num binary tests:  4
--Num binary stages:  4
Setting stages to  6
Setting stages to  7

TESTING group:  [0, 0, 0, 0]
0 infected --> ran only 1 test
Setting stages to  7


(19, 7)

In [27]:
m_arr4 = [0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
lisa(m_arr4)

RUNNING TEST ON original:  [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 


TESTING group:  [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

TESTING group:  [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]

TESTING group:  [0, 0, 0, 0, 1, 0, 0]
--Num binary tests:  5
--Num binary stages:  5
TRYING STAGES: setting to  6

TESTING group:  [0, 0, 1, 0, 0, 0, 0]
--Num binary tests:  5
--Num binary stages:  5
TRYING STAGES: setting to  6
TRYING STAGES: setting to  6

TESTING group:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0 infected --> ran only 1 test
TRYING STAGES: setting to  6


(15, 6)

In [9]:

#Returns a quantized number depending on infections
#0 
#1 (1<= I < 2)
#2 (2 <= I < 4)
#3 (4 <= I < 8)
#4 (>= 8)
def num_infected2(s):
    inf_people = 0
    for i in range(len(s)):
        inf_people += s[i]
    
    if(inf_people == 0):
        return 0
    elif(inf_people == 1):
        return 1
    elif(inf_people == 2 or inf_people == 3):
        return 2
    elif (inf_people < 8):
        return 3
    else:
        return 4
    

In [10]:
def pablo(s, max_stages):
    inf_range = num_infected2(s)
    print("\nTESTING group: ", s)
  
    if (inf_range == 0):
        print("0 infected --> ran only 1 test")
        return 1, 1
    elif (inf_range  == 1):
        # Only 1 infected person
        if len(s) == 1:
            return 1,1
        else:
            binary_tests, binary_stages, _ = binary_splitting(s)
            print("--Num binary tests: ", binary_tests)
            print("--Num binary stages: ", binary_stages)
            return 1 + binary_tests, 1 + binary_stages
   
    inf_people = 0
    if (inf_range == 2):
        inf_people = 3
    elif (inf_range == 3):
        inf_people = 7
    else:
        inf_people = 8

    num_per_group = round(len(s)/inf_people)
        
        # for each subgroup, call function again
    tests = 1 # = this testing group
    for i in range(inf_people):
        stages = 1
        if(i == inf_people-1):
            subGroup = s[i*num_per_group:]
        else:
            subGroup = s[i*num_per_group:(i+1)*(num_per_group)]
            
        t, st = pablo(subGroup, max_stages)
        stages += st
        tests += t   # number of tests in subtree

        # delete:
        m = max(max_stages, stages)
        if (m == stages and m != max_stages):
            print("Updating max stages from ", max_stages, " to ", stages)

        max_stages = max(max_stages, stages)
    return tests, max_stages
    

def picasso(s):
    '''
    s(np.array): binary string of infection status
    '''
    num_tests = 0
    stages = 0
    ###################################################
    '''your code here'''
    print("RUNNING TEST ON original: ", s, "\n")
    num_tests, stages = pablo(s, stages)
    ###################################################

    return num_tests,stages


m_arr3 = [0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0]
picasso(m_arr3)

RUNNING TEST ON original:  [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0] 


TESTING group:  [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]

TESTING group:  [0, 0, 0]
0 infected --> ran only 1 test
Updating max stages from  0  to  2

TESTING group:  [0, 1, 0]
--Num binary tests:  4
--Num binary stages:  4
Updating max stages from  2  to  6

TESTING group:  [0, 0, 0]
0 infected --> ran only 1 test

TESTING group:  [1, 0, 1]

TESTING group:  [1]

TESTING group:  [0]
0 infected --> ran only 1 test

TESTING group:  [1]
Updating max stages from  6  to  7

TESTING group:  [0, 1, 0]
--Num binary tests:  4
--Num binary stages:  4

TESTING group:  [0, 0, 0]
0 infected --> ran only 1 test

TESTING group:  [0]
0 infected --> ran only 1 test


(19, 7)

In [44]:
m_arr3 = [0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0]
picasso(m_arr3)

RUNNING TEST ON original:  [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0] 


TESTING group:  [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]

TESTING group:  [0, 0, 0]
0 infected --> ran only 1 test
Updating max stages from  0  to  2

TESTING group:  [0, 1, 0]
--Num binary tests:  4
--Num binary stages:  4
Updating max stages from  2  to  6

TESTING group:  [0, 0, 0]
0 infected --> ran only 1 test

TESTING group:  [1, 0, 1]

TESTING group:  [1]

TESTING group:  [0]
0 infected --> ran only 1 test

TESTING group:  [1]
Updating max stages from  6  to  7

TESTING group:  [0, 1, 0]
--Num binary tests:  4
--Num binary stages:  4

TESTING group:  [0, 0, 0]
0 infected --> ran only 1 test

TESTING group:  [0]
0 infected --> ran only 1 test


(19, 7)